*This code is written by Yohan.(https://www.kaggle.com/yohanb/categorical-features-encoding-xgb-0-554) Thanks for sharing!

### 이 노트북에서는 범주형 features를 인코딩해 볼 것입니다.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import xgboost as xgb 
from sklearn.metrics import r2_score

from IPython.display import display, HTML
# Shows all columns of a dataframe
def show_dataframe(X, rows = 2):
    display(HTML(X.to_html(max_rows=rows)))